In [7]:
import os
# os.environ['ALFRED_ROOT'] = '/root/data/home/hoyeung/alfred/'
os.environ['ALFRED_ROOT'] = '/home/legg/alfred/'

import sys
sys.path.append(os.path.join(os.environ['ALFRED_ROOT']))
sys.path.append(os.path.join(os.environ['ALFRED_ROOT'], 'models'))

# from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
import torch
import pprint
import json
import numpy as np
from data.preprocess import Dataset
from importlib import import_module, reload
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
from collections import Counter
# from models.utils.helper_utils import optimizer_to

In [2]:
parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter)
args = parser.parse_args('')

# settings
# args.data = '/root/data_alfred/json_data_augmentation_20200820'
args.data = '/media/legg/data/alfred_data/json_data_augmentation_20200820'
args.pp_folder = 'pp'
args.pframe = 300
args.fast_epoch = True

In [3]:
vocab = torch.load(os.path.join('/media/legg/data/alfred_data/json_feat_2.1.0_backup_20200826_agent_training', "%s.vocab" % args.pp_folder))

In [4]:
vocab

{'word': Vocab(2360), 'action_low': Vocab(15), 'action_high': Vocab(93)}

## Preprocess original train splits oct21

In [5]:
# SKIP preprocessing. Already preprocessed

## Preprocess augmented splits

In [26]:
args.splits = '/root/data_alfred/splits/sample_failed_20200820_filtered.json'
# args.splits = '/root/data_alfred/splits/sample_failed_toy.json'

with open(args.splits, 'r') as f:
    splits = json.load(f)

print([(k,len(splits[k])) for k in splits.keys()])

[('augmentation', 7570)]


In [27]:
splits['augmentation'][0]

{'task': 'look_at_obj_in_light-BaseballBat-None-DeskLamp-301/trial_T20200814_164125_595727',
 'repeat_idx': 0,
 'full_traj_success': False,
 'collected_subgoals': 2}

In [28]:
args.fast_epoch = False
dataset = Dataset(args, vocab)
dataset.preprocess_splits_augmentation(splits, baseline=True)

100% (7570 of 7570) |####################| Elapsed Time: 0:01:38 Time:  0:01:38


In [29]:
args.fast_epoch = False
dataset = Dataset(args, vocab)
dataset.preprocess_splits_augmentation(splits, baseline=False)

100% (7570 of 7570) |####################| Elapsed Time: 0:01:29 Time:  0:01:29


# Make SPLITS

## Make Toy Training + Augmentation Split

In [13]:
alfred_splits_p = '/root/data_alfred/splits/oct21.json'
with open(alfred_splits_p, 'r') as f:
    alfred_splits = json.load(f)
print([(k,len(alfred_splits[k])) for k in alfred_splits.keys()])

[('tests_seen', 1533), ('tests_unseen', 1529), ('train', 21023), ('valid_seen', 820), ('valid_unseen', 821)]


In [14]:
toy_aug_splits_p = '/root/data_alfred/splits/sample_failed_toy.json'
with open(toy_aug_splits_p, 'r') as f:
    toy_aug_splits = json.load(f)

print([(k,len(toy_aug_splits[k])) for k in toy_aug_splits.keys()])

[('augmentation', 40)]


In [16]:
toy_agent_tr_splits = {}

for k in alfred_splits.keys():
    toy_agent_tr_splits[k] = alfred_splits[k][:16]

toy_agent_tr_splits['augmentation'] = toy_aug_splits['augmentation']

print([(k,len(toy_agent_tr_splits[k])) for k in toy_agent_tr_splits.keys()])

[('tests_seen', 16), ('tests_unseen', 16), ('train', 16), ('valid_seen', 16), ('valid_unseen', 16), ('augmentation', 40)]


In [17]:
toy_split_p = '/root/data_alfred/splits/agent_augmentation_20200825.json'

with open(toy_split_p, 'w') as f:
    json.dump(toy_agent_tr_splits, f)

In [18]:
with open(toy_split_p, 'r') as f:
    dum = json.load(f)

In [19]:
dum

{'tests_seen': [{'repeat_idx': 0, 'task': 'trial_T20190909_042500_949430'},
  {'repeat_idx': 1, 'task': 'trial_T20190909_042500_949430'},
  {'repeat_idx': 2, 'task': 'trial_T20190909_042500_949430'},
  {'repeat_idx': 0, 'task': 'trial_T20190908_064522_624362'},
  {'repeat_idx': 1, 'task': 'trial_T20190908_064522_624362'},
  {'repeat_idx': 2, 'task': 'trial_T20190908_064522_624362'},
  {'repeat_idx': 0, 'task': 'trial_T20190909_085207_886285'},
  {'repeat_idx': 1, 'task': 'trial_T20190909_085207_886285'},
  {'repeat_idx': 2, 'task': 'trial_T20190909_085207_886285'},
  {'repeat_idx': 0, 'task': 'trial_T20190908_132517_128346'},
  {'repeat_idx': 1, 'task': 'trial_T20190908_132517_128346'},
  {'repeat_idx': 2, 'task': 'trial_T20190908_132517_128346'},
  {'repeat_idx': 0, 'task': 'trial_T20190908_133706_349398'},
  {'repeat_idx': 1, 'task': 'trial_T20190908_133706_349398'},
  {'repeat_idx': 2, 'task': 'trial_T20190908_133706_349398'},
  {'repeat_idx': 0, 'task': 'trial_T20190911_082512_6786

# Make SPLITS

## Read in existing splits

In [5]:
original_alfred_splits_p = '/media/legg/data/alfred_data/splits/oct21.json'

with open(original_alfred_splits_p, 'r') as f:
    original_alfred_splits = json.load(f)
    
print([(k,len(original_alfred_splits[k])) for k in original_alfred_splits.keys()])

[('tests_seen', 1533), ('tests_unseen', 1529), ('train', 21023), ('valid_seen', 820), ('valid_unseen', 821)]


In [32]:
augmented_splits_p = '/media/legg/data/alfred_data/splits/sample_failed_20200820_filtered.json'

with open(augmented_splits_p, 'r') as f:
    augmented_splits = json.load(f)

print([(k,len(augmented_splits[k])) for k in augmented_splits.keys()])

[('augmentation', 7570)]


## 1. Annotate all failures with Explainer and with Baseline.  Retrain agent and evaluate

In [37]:
experiment_1_splits = {}

for k in original_alfred_splits.keys():
    experiment_1_splits[k] = original_alfred_splits[k] 

experiment_1_splits['augmentation'] = augmented_splits['augmentation']

print([(k,len(experiment_1_splits[k])) for k in experiment_1_splits.keys()])

[('tests_seen', 1533), ('tests_unseen', 1529), ('train', 21023), ('valid_seen', 820), ('valid_unseen', 821), ('augmentation', 7570)]


In [39]:
experiment_1_split_path = '/root/data_alfred/splits/data_augmentation_experiment1_20200826.json'
with open(experiment_1_split_path, 'w') as f:
    json.dump(experiment_1_splits, f)
    
with open(experiment_1_split_path, 'r') as f:
    dum = json.load(f)

print([(k,len(dum[k])) for k in dum.keys()])

[('tests_seen', 1533), ('tests_unseen', 1529), ('train', 21023), ('valid_seen', 820), ('valid_unseen', 821), ('augmentation', 7570)]


## 2. Retrain agent on only half the original training data and evaluate 

In [8]:
# distribution -- number of annotations per task
counter = dict()

for task in original_alfred_splits['train']:
    if task['task'] not in counter:
        counter[task['task']] = 0
    counter[task['task']] += 1
    
print(Counter(counter.values()))
print(sum([k*v for k,v in Counter(counter.values()).items()]))
print(len(counter.keys()))

Counter({3: 5990, 6: 282, 5: 156, 4: 145, 1: 1})
21023
6574


In [9]:
# randomly select half of the tasks
np.random.seed(42)
keep_tasks = np.random.choice(list(counter.keys()), size=len(counter)//2, replace=False)
print(keep_tasks[:10])
keep_tasks = set(keep_tasks)
print(len(keep_tasks))

['pick_and_place_simple-WineBottle-None-Shelf-7/trial_T20190907_200104_945677'
 'pick_cool_then_place_in_recep-Bowl-None-Cabinet-24/trial_T20190909_112909_923737'
 'pick_heat_then_place_in_recep-AppleSliced-None-DiningTable-17/trial_T20190908_095621_618627'
 'pick_and_place_simple-TennisRacket-None-SideTable-310/trial_T20190907_040920_796378'
 'pick_two_obj_and_place-CreditCard-None-Desk-313/trial_T20190909_104526_715846'
 'pick_and_place_simple-Candle-None-Cart-430/trial_T20190909_055319_119903'
 'pick_heat_then_place_in_recep-PotatoSliced-None-Fridge-22/trial_T20190908_185723_946017'
 'pick_and_place_with_movable_recep-RemoteControl-Box-Sofa-220/trial_T20190907_023003_992808'
 'pick_heat_then_place_in_recep-Egg-None-SinkBasin-25/trial_T20190907_024723_698757'
 'pick_cool_then_place_in_recep-Plate-None-Cabinet-25/trial_T20190908_102731_668233']
3287


In [10]:
# create new train split
half_train_split = []
ct = 0
for task in original_alfred_splits['train']:
    if task['task'] in keep_tasks:
        ct += 1
        half_train_split.append(task)
        
print(ct)

10524


In [147]:
# assemble splits
experiment_2_splits = {}

for k in original_alfred_splits.keys():
    if k != 'train':
        experiment_2_splits[k] = original_alfred_splits[k] 

experiment_2_splits['train'] = half_train_split

print([(k,len(experiment_2_splits[k])) for k in experiment_2_splits.keys()])

[('tests_seen', 1533), ('tests_unseen', 1529), ('valid_seen', 820), ('valid_unseen', 821), ('train', 10524)]


In [151]:
# save out
experiment_2_split_path = '/media/legg/data/alfred_data/splits/data_augmentation_experiment2_20200831.json'

with open(experiment_2_split_path, 'w') as f:
    json.dump(experiment_2_splits, f)
    
with open(experiment_2_split_path, 'r') as f:
    dum = json.load(f)

print([(k,len(dum[k])) for k in dum.keys()])

[('tests_seen', 1533), ('tests_unseen', 1529), ('valid_seen', 820), ('valid_unseen', 821), ('train', 10524)]


## 3. Label the other half of the training data with the Explainer, retrain agent, eval

In [12]:
# create new train split
other_half_train_split = []
ct = 0
for task in original_alfred_splits['train']:
    if task['task'] not in keep_tasks:
        ct += 1
        task['use_autogeneration'] = True
        other_half_train_split.append(task)
        
print(ct)

10499


In [19]:
print([(k,len(original_alfred_splits[k])) for k in original_alfred_splits.keys()])

[('tests_seen', 1533), ('tests_unseen', 1529), ('train', 21023), ('valid_seen', 820), ('valid_unseen', 821)]


In [24]:
# save out
experiment_3_split_path = '/media/legg/data/alfred_data/splits/data_augmentation_experiment3_20200901.json'

# original_alfred_splits has been modified inplace
with open(experiment_3_split_path, 'w') as f:
    json.dump(original_alfred_splits, f)
    
with open(experiment_3_split_path, 'r') as f:
    dum = json.load(f)

print([(k,len(dum[k])) for k in dum.keys()])

[('tests_seen', 1533), ('tests_unseen', 1529), ('train', 21023), ('valid_seen', 820), ('valid_unseen', 821)]


In [25]:
original_alfred_splits['train']

[{'repeat_idx': 0,
  'task': 'pick_cool_then_place_in_recep-LettuceSliced-None-DiningTable-17/trial_T20190909_070538_437648',
  'use_autogeneration': True},
 {'repeat_idx': 1,
  'task': 'pick_cool_then_place_in_recep-LettuceSliced-None-DiningTable-17/trial_T20190909_070538_437648',
  'use_autogeneration': True},
 {'repeat_idx': 2,
  'task': 'pick_cool_then_place_in_recep-LettuceSliced-None-DiningTable-17/trial_T20190909_070538_437648',
  'use_autogeneration': True},
 {'repeat_idx': 0,
  'task': 'pick_two_obj_and_place-Candle-None-Cabinet-414/trial_T20190908_190650_163902',
  'use_autogeneration': True},
 {'repeat_idx': 1,
  'task': 'pick_two_obj_and_place-Candle-None-Cabinet-414/trial_T20190908_190650_163902',
  'use_autogeneration': True},
 {'repeat_idx': 2,
  'task': 'pick_two_obj_and_place-Candle-None-Cabinet-414/trial_T20190908_190650_163902',
  'use_autogeneration': True},
 {'repeat_idx': 0,
  'task': 'pick_heat_then_place_in_recep-Egg-None-SinkBasin-20/trial_T20190908_205050_0009

In [36]:
p = '/media/legg/data/alfred_data/json_feat_2.1.0_backup_20200826_agent_training'
# data_p = os.path.join(p, 'pick_cool_then_place_in_recep-LettuceSliced-None-DiningTable-17/trial_T20190909_070538_437648/pp/', 'aug_explainer_0_T20200810_1829.json')
data_p = os.path.join(p, 'pick_cool_then_place_in_recep-LettuceSliced-None-DiningTable-17/trial_T20190909_070538_437648/pp/', 'ann_0.json')


with open(data_p, 'r') as f:
    data1 = json.load(f)

In [37]:
p = '/media/legg/data/alfred_data/json_feat_2.1.0_backup_20200826_agent_training'
data_p = os.path.join(p, 'pick_cool_then_place_in_recep-LettuceSliced-None-DiningTable-17/trial_T20190909_070538_437648/pp/', 'aug_explainer_0_T20200810_1829.json')
# data_p = os.path.join(p, 'pick_cool_then_place_in_recep-LettuceSliced-None-DiningTable-17/trial_T20190909_070538_437648/pp/', 'ann_0.json')


with open(data_p, 'r') as f:
    data2 = json.load(f)